In [50]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [51]:
resp = requests.get("https://en.wikipedia.org/wiki/List_of_prominent_mountains_of_the_Alps_(2000%E2%80%932499_m)")

In [52]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [53]:
tables = soup.findAll("table")

In [55]:
## googlato "python parse table from html" -> https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/ e copiato il codice XD

import requests
import pandas as pd
from bs4 import BeautifulSoup

class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                    
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
        
        return df

In [57]:
hp = HTMLTableParser()

parsed_table = hp.parse_html_table(tables[1])

In [59]:
parsed_table[:50]

,Rank[Note 1]\n,Mountain\n,Height(m)[1]\n,Drop (m)[Note 2]\n,Coordinates[2]\n,Range [Note 3]\n,Range[Note 4]\n,Region [Note 5]\n,Country\n
0,1052\n,Marchkopf\n,2499.0,372.0,".mw-parser-output .geo-default,.mw-parser-outp...",Tux Alps\n,II/B-23.I-B\n,North Tyrol\n,AT\n
1,1053\n,Castello di Moschesin\n,2499.0,314.0,46°17′50″N 12°07′58″E﻿ / ﻿46.29722°N 12.13278°...,Dolomites – SE\n,II/C-31.II-B\n,Belluno\n,IT\n
2,1054\n,Monte Croce [it]\n,2496.0,533.0,46°10′10″N 11°23′32″E﻿ / ﻿46.16944°N 11.39222°...,Fiemme Mountains\n,II/C-31.V-B\n,Trentino\n,IT\n
3,1055\n,Mittlere Kreuzspitze [de]\n,2496.0,456.0,47°20′10″N 10°35′34″E﻿ / ﻿47.33611°N 10.59278°...,Lechtal Alps\n,II/B-21.I-B\n,North Tyrol\n,AT\n
4,1056\n,Les Grands Moulins\n,2495.0,307.0,45°24′26″N 06°11′54″E﻿ / ﻿45.40722°N 6.19833°E...,Belledonne\n,I/A-05.II.B\n,Savoie\n,FR\n
5,1057\n,Wannig\n,2493.0,704.0,47°20′13″N 10°51′44″E﻿ / ﻿47.33694°N 10.86222°...,Mieminger Chain\n,II/B-21.III-A\n,North Tyrol\n,AT\n
6,1058\n,Pizzo Camino\n,2491.0,525.0,45°58′54″N 10°10′39″E﻿ / ﻿45.98167°N 10.17750°...,Bergamasque Prealps\n,II/C-29.II-C\n,Bergamo / Brescia\n,IT\n
7,1059\n,Fulen\n,2491.0,311.0,46°55′07″N 08°42′53″E﻿ / ﻿46.91861°N 8.71472°E...,Schwyz Alps\n,I/B-14.IV-A\n,Schwyz / Uri\n,CH\n
8,1060\n,Viezzena [it]\n,2490.0,435.0,46°20′15″N 11°40′32″E﻿ / ﻿46.33750°N 11.67556°...,Dolomites – NW\n,II/C-31.III-B\n,South Tyrol\n,IT\n
9,1061\n,Zanggen / Pala di Santa [de]\n,2488.0,498.0,46°20′45″N 11°30′43″E﻿ / ﻿46.34583°N 11.51194°...,Fiemme Mountains\n,II/C-31.V-A\n,South Tyrol / Trentino\n,IT\n
